In [718]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import math

from scipy import stats

In [719]:
shark_attacks = pd.read_csv("GSAF5.xls.csv")
print(len(shark_attacks))
print(shark_attacks.dtypes)

# Note that columns 10, 17, 18, 19, 20, 21, 24 have "mixed types".
# Also note that from column 22 and onwards, there are extra columns that have no data up to column 255.
# 10 = Unnamed: 9 (M or F or empty)
# 17 = pdf
# 18 = href formula
# 19 = href
# 20 = Case Number.1
# 21 = Case Number.2
# 24 = Unnamed:23 (seems to be a total of 2-3 random notes from a data cataloguer)

6462
index             int64
Case Number      object
Date             object
Year            float64
Type             object
                 ...   
Unnamed: 251    float64
Unnamed: 252    float64
Unnamed: 253    float64
Unnamed: 254    float64
Unnamed: 255    float64
Length: 257, dtype: object


C:\Users\skyli\AppData\Local\Temp\ipykernel_12400\2291830759.py:1: DtypeWarning: Columns (10,17,18,19,20,21,24) have mixed types. Specify dtype option on import or set low_memory=False.
  shark_attacks = pd.read_csv("GSAF5.xls.csv")


In [720]:
# Step 1: Remove bogus columns.

shark_attacks = shark_attacks[["index", "Case Number", "Date", "Year", "Type", "Country", "Area", "Location", "Activity", "Name", "Unnamed: 9", "Age", "Injury", "Fatal (Y/N)", "Time", "Species ", "Investigator or Source", "pdf", "href formula", "href", "Case Number.1", "Case Number.2", "original order"]]

shark_attacks

,index,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,0,2020.02.05,5-Feb-20,2020.0,Unprovoked,USA,Maui,NaN,Stand-Up Paddle boarding,NaN,...,N,09h40,Tiger shark,"K. McMurray, TrackingSharks.com",NaN,NaN,NaN,NaN,NaN,NaN
1,1,2020.01.30.R,Reported 30-Jan-2020,2020.0,Provoked,BAHAMAS,Exumas,NaN,Floating,Ana Bruna Avila,...,N,NaN,NaN,"K. McMurray, TrackingSharks.com",NaN,NaN,NaN,NaN,NaN,NaN
2,2,2020.01.17,17-Jan-20,2020.0,Unprovoked,AUSTRALIA,New South Wales,Windang Beach,Surfing,Will Schroeter,...,N,08h00,"""A small shark""","B. Myatt & M. Michaelson, GSAF; K. McMurray, T...",2020.01.17-Schroeter.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.01.17,2020.01.17,6502.0
3,3,2020.01.16,16-Jan-20,2020.0,Unprovoked,NEW ZEALAND,Southland,Oreti Beach,Surfing,Jordan King,...,N,20h30,Broadnose seven gill shark?,"K. McMurray, TrackingSharks.com",2020.01.16-King.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.01.16,2020.01.16,6501.0
4,4,2020.01.13,13-Jan-20,2020.0,Unprovoked,USA,North Carolina,"Rodanthe, Dare County",Surfing,Samuel Horne,...,N,14h33,NaN,"C. Creswell, GSAF",2020.01.13-Horne.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.01.13,2020.01.13,6500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6457,6457,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,...,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",NaN,NaN,NaN,NaN,NaN,NaN
6458,6458,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,...,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",NaN,NaN,NaN,NaN,NaN,NaN
6459,6459,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,...,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",NaN,NaN,NaN,NaN,NaN,NaN
6460,6460,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,...,Y,NaN,NaN,"The Sun, 10/20/1938",NaN,NaN,NaN,NaN,NaN,NaN


In [721]:
# Step 2: Remove irrelevant columns for  ("Investigator or Source", "Case Number", "Name", "Case Number.1", "Case Number.2", "pdf", "href formula", "href", "original order").

shark_attacks = shark_attacks[["index", "Date", "Year", "Type", "Country", "Area", "Location", "Activity", "Unnamed: 9", "Age", "Injury", "Fatal (Y/N)", "Time", "Species "]]

shark_attacks

,index,Date,Year,Type,Country,Area,Location,Activity,Unnamed: 9,Age,Injury,Fatal (Y/N),Time,Species
0,0,5-Feb-20,2020.0,Unprovoked,USA,Maui,NaN,Stand-Up Paddle boarding,NaN,20s,"No injury, but paddleboard bitten",N,09h40,Tiger shark
1,1,Reported 30-Jan-2020,2020.0,Provoked,BAHAMAS,Exumas,NaN,Floating,F,24,PROVOKED INCIDENT Scratches to left wrist,N,NaN,NaN
2,2,17-Jan-20,2020.0,Unprovoked,AUSTRALIA,New South Wales,Windang Beach,Surfing,M,59,Laceration ot left ankle and foot,N,08h00,"""A small shark"""
3,3,16-Jan-20,2020.0,Unprovoked,NEW ZEALAND,Southland,Oreti Beach,Surfing,F,13,Minor injury to lower leg,N,20h30,Broadnose seven gill shark?
4,4,13-Jan-20,2020.0,Unprovoked,USA,North Carolina,"Rodanthe, Dare County",Surfing,M,26,Lacerations to foot,N,14h33,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6457,6457,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,NaN,NaN,FATAL,Y,NaN,NaN
6458,6458,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,NaN,NaN,FATAL,Y,NaN,NaN
6459,6459,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,NaN,NaN,FATAL,Y,NaN,NaN
6460,6460,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,NaN,NaN,FATAL,Y,NaN,NaN


In [722]:
# Step 3: Clean "Age" column to include those with known ages.

def clean_age(age):

    sage = str(age)  # Transform age to a string for easier verification.

    # If the age contains more than just digits...

    if not sage.isdigit():

        # If the age ends with "s", replace last digit with a "5" and remove the "s" (e.g. assume "30s" is "35", being the average between 30 and 40).

        if sage[-1] == "s":

            lage = list(sage)  # Transform the string into a list in order to do inplace editing.

            lage[-2] = "5"
            lage.pop()
            
            return "".join(lage)  # Return the list as a string.

        return sage  # No other operation performed for cleaning.

    return sage  # Was already clean.

shark_attacks["Age"] = shark_attacks["Age"].apply(clean_age)  # Modify values to be acceptable, where possible.
shark_attacks = shark_attacks[shark_attacks["Age"].str.match("^[0-9]+$")]  # Reduce matches to only those ages that are exact numbers.
shark_attacks

C:\Users\skyli\AppData\Local\Temp\ipykernel_12400\2609986971.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_attacks["Age"] = shark_attacks["Age"].apply(clean_age)  # Modify values to be acceptable, where possible.


,index,Date,Year,Type,Country,Area,Location,Activity,Unnamed: 9,Age,Injury,Fatal (Y/N),Time,Species
0,0,5-Feb-20,2020.0,Unprovoked,USA,Maui,NaN,Stand-Up Paddle boarding,NaN,25,"No injury, but paddleboard bitten",N,09h40,Tiger shark
1,1,Reported 30-Jan-2020,2020.0,Provoked,BAHAMAS,Exumas,NaN,Floating,F,24,PROVOKED INCIDENT Scratches to left wrist,N,NaN,NaN
2,2,17-Jan-20,2020.0,Unprovoked,AUSTRALIA,New South Wales,Windang Beach,Surfing,M,59,Laceration ot left ankle and foot,N,08h00,"""A small shark"""
3,3,16-Jan-20,2020.0,Unprovoked,NEW ZEALAND,Southland,Oreti Beach,Surfing,F,13,Minor injury to lower leg,N,20h30,Broadnose seven gill shark?
4,4,13-Jan-20,2020.0,Unprovoked,USA,North Carolina,"Rodanthe, Dare County",Surfing,M,26,Lacerations to foot,N,14h33,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6402,6402,1960s,0.0,Unprovoked,IRAQ,Basrah,Shatt-al-Arab River,Swimming naked near a date palm where many dat...,NaN,6,Arm severed,N,Afternoon,Bull shark
6403,6403,1960s,0.0,Unprovoked,IRAQ,Basrah,Shatt-al-Arab River near Abu al Khasib,Swimming in section of river used for washing ...,NaN,16,Right leg lacerated & surgically amputated,N,Afternoon,Bull shark
6414,6414,Before 1957,0.0,Provoked,CUBA,Havana Province,Cojimar,"Shark fishing, knocked overboard",NaN,50,"FATAL, hip bitten PROVOKED INCIDENT",Y,NaN,NaN
6436,6436,Early 1930s,0.0,Unprovoked,BELIZE,NaN,NaN,Standing,NaN,16,FATAL,Y,NaN,12' tiger shark


In [723]:
# Step 4: Clean "Date" column to have a uniform date. Exclude rows where no exact date is known.

def clean_date(date):

    # Transform NaN into strings, to be ejected later.

    if not isinstance(date, str):
        return ""
    
    re_match = re.match("^(Reported )?([0-9]+)(/|-|\.)([0-9a-zA-Z]+)(/|-|\.)+([0-9]+)$", date)

    # If there was a match for the generally-expected input...
    
    if re_match != None:

        day = re_match.group(2)
        month = re_match.group(4)
        year = re_match.group(6)

        # If the day is 4 characters long, format input was year.month.day, so swap the year and day.

        if  len(day) == 4:
            daytemp = day
            day = year
            year = daytemp

        # If the year was only 2 characters long, assume the year to be 2020.
            
        if len(year) == 2:
            year = '20' + year

        # If the month was not numeric, make it so.

        if not month.isdigit():
            if month == 'Jan':
                month = '1'
            elif month == 'Feb':
                month = '2'
            elif month == 'Mar':
                month = '3'
            elif month == 'Apr':
                month = '4'
            elif month == 'May':
                month = '5'
            elif month == 'Jun':
                month = '6'
            elif month == 'Jul':
                month = '7'
            elif month == 'Aug':
                month = '8'
            elif month == 'Sep':
                month = '9'
            elif month == 'Oct':
                month = '10'
            elif month == 'Nov':
                month = '11'
            elif month == 'Dec':
                month = '12'

        # Return a normalized date.

        return month + '/' + day + '/' + year

    # Did not match expected format, set to empty (to mark it for deletion after).

    return ""

shark_attacks["Date"] = shark_attacks["Date"].apply(clean_date)
shark_attacks = shark_attacks[~shark_attacks["Date"].str.match("^$")]  # Remove columns that became empty as a result of cleaning.

shark_attacks

C:\Users\skyli\AppData\Local\Temp\ipykernel_12400\296291635.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_attacks["Date"] = shark_attacks["Date"].apply(clean_date)


,index,Date,Year,Type,Country,Area,Location,Activity,Unnamed: 9,Age,Injury,Fatal (Y/N),Time,Species
0,0,2/5/2020,2020.0,Unprovoked,USA,Maui,NaN,Stand-Up Paddle boarding,NaN,25,"No injury, but paddleboard bitten",N,09h40,Tiger shark
1,1,1/30/2020,2020.0,Provoked,BAHAMAS,Exumas,NaN,Floating,F,24,PROVOKED INCIDENT Scratches to left wrist,N,NaN,NaN
2,2,1/17/2020,2020.0,Unprovoked,AUSTRALIA,New South Wales,Windang Beach,Surfing,M,59,Laceration ot left ankle and foot,N,08h00,"""A small shark"""
3,3,1/16/2020,2020.0,Unprovoked,NEW ZEALAND,Southland,Oreti Beach,Surfing,F,13,Minor injury to lower leg,N,20h30,Broadnose seven gill shark?
4,4,1/13/2020,2020.0,Unprovoked,USA,North Carolina,"Rodanthe, Dare County",Surfing,M,26,Lacerations to foot,N,14h33,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6237,6237,1/17/1837,1837.0,Unprovoked,AUSTRALIA,New South Wales,Macleay River,Washing his feet,NaN,12,"FATAL Injured by shark, died of tetanus",Y,Evening,NaN
6249,6249,7/26/1830,1830.0,Unprovoked,USA,Massachusetts,"Swampscott, Essex County","Fishing from dory, shark upset boat & he fell ...",NaN,52,FATAL,Y,NaN,NaN
6256,6256,9/28/1828,1828.0,Unprovoked,SIERRA LEONE,Western Area,"River Sierra Leone, 35 miles upriver from Free...","British ship, Britannia, was loading lumber. H...",NaN,17,"Left arm severed 2.5"" from elbow, groin, abdom...",N,NaN,NaN
6274,6274,5/11/1817,1817.0,Unprovoked,SRI LANKA,Western Province,Colombo,Swimming,NaN,22,FATAL,Y,Evening,NaN


In [724]:
# Step 5: Split the "Date" column up into 3 new columns: "Day", "Month", and "Year".

shark_attacks[['Month', 'Day', 'Year']] = shark_attacks['Date'].str.split('/', expand=True)
shark_attacks = shark_attacks.drop('Date', axis=1)  # "Date" column no longer needed.

shark_attacks

C:\Users\skyli\AppData\Local\Temp\ipykernel_12400\1218471878.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_attacks[['Month', 'Day', 'Year']] = shark_attacks['Date'].str.split('/', expand=True)
C:\Users\skyli\AppData\Local\Temp\ipykernel_12400\1218471878.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_attacks[['Month', 'Day', 'Year']] = shark_attacks['Date'].str.split('/', expand=True)


,index,Year,Type,Country,Area,Location,Activity,Unnamed: 9,Age,Injury,Fatal (Y/N),Time,Species,Month,Day
0,0,2020,Unprovoked,USA,Maui,NaN,Stand-Up Paddle boarding,NaN,25,"No injury, but paddleboard bitten",N,09h40,Tiger shark,2,5
1,1,2020,Provoked,BAHAMAS,Exumas,NaN,Floating,F,24,PROVOKED INCIDENT Scratches to left wrist,N,NaN,NaN,1,30
2,2,2020,Unprovoked,AUSTRALIA,New South Wales,Windang Beach,Surfing,M,59,Laceration ot left ankle and foot,N,08h00,"""A small shark""",1,17
3,3,2020,Unprovoked,NEW ZEALAND,Southland,Oreti Beach,Surfing,F,13,Minor injury to lower leg,N,20h30,Broadnose seven gill shark?,1,16
4,4,2020,Unprovoked,USA,North Carolina,"Rodanthe, Dare County",Surfing,M,26,Lacerations to foot,N,14h33,NaN,1,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6237,6237,1837,Unprovoked,AUSTRALIA,New South Wales,Macleay River,Washing his feet,NaN,12,"FATAL Injured by shark, died of tetanus",Y,Evening,NaN,1,17
6249,6249,1830,Unprovoked,USA,Massachusetts,"Swampscott, Essex County","Fishing from dory, shark upset boat & he fell ...",NaN,52,FATAL,Y,NaN,NaN,7,26
6256,6256,1828,Unprovoked,SIERRA LEONE,Western Area,"River Sierra Leone, 35 miles upriver from Free...","British ship, Britannia, was loading lumber. H...",NaN,17,"Left arm severed 2.5"" from elbow, groin, abdom...",N,NaN,NaN,9,28
6274,6274,1817,Unprovoked,SRI LANKA,Western Province,Colombo,Swimming,NaN,22,FATAL,Y,Evening,NaN,5,11


In [725]:
# Step 6: Remove rows where the type of attack is "Under Investigation" (don't know if it's a shark attack) or "Invalid" (it was not a shark attack).

def clean_type(type):
    if not isinstance(type, str):
        return ""
    return type

shark_attacks["Type"] = shark_attacks["Type"].apply(clean_type)
shark_attacks = shark_attacks[~shark_attacks["Type"].str.match("^Invalid$")]

shark_attacks

,index,Year,Type,Country,Area,Location,Activity,Unnamed: 9,Age,Injury,Fatal (Y/N),Time,Species,Month,Day
0,0,2020,Unprovoked,USA,Maui,NaN,Stand-Up Paddle boarding,NaN,25,"No injury, but paddleboard bitten",N,09h40,Tiger shark,2,5
1,1,2020,Provoked,BAHAMAS,Exumas,NaN,Floating,F,24,PROVOKED INCIDENT Scratches to left wrist,N,NaN,NaN,1,30
2,2,2020,Unprovoked,AUSTRALIA,New South Wales,Windang Beach,Surfing,M,59,Laceration ot left ankle and foot,N,08h00,"""A small shark""",1,17
3,3,2020,Unprovoked,NEW ZEALAND,Southland,Oreti Beach,Surfing,F,13,Minor injury to lower leg,N,20h30,Broadnose seven gill shark?,1,16
4,4,2020,Unprovoked,USA,North Carolina,"Rodanthe, Dare County",Surfing,M,26,Lacerations to foot,N,14h33,NaN,1,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6237,6237,1837,Unprovoked,AUSTRALIA,New South Wales,Macleay River,Washing his feet,NaN,12,"FATAL Injured by shark, died of tetanus",Y,Evening,NaN,1,17
6249,6249,1830,Unprovoked,USA,Massachusetts,"Swampscott, Essex County","Fishing from dory, shark upset boat & he fell ...",NaN,52,FATAL,Y,NaN,NaN,7,26
6256,6256,1828,Unprovoked,SIERRA LEONE,Western Area,"River Sierra Leone, 35 miles upriver from Free...","British ship, Britannia, was loading lumber. H...",NaN,17,"Left arm severed 2.5"" from elbow, groin, abdom...",N,NaN,NaN,9,28
6274,6274,1817,Unprovoked,SRI LANKA,Western Province,Colombo,Swimming,NaN,22,FATAL,Y,Evening,NaN,5,11


In [726]:
# Step 7: Remove rows where the "Country" and "Location" is empty or inexact.

def clean_empty(data):

    if not isinstance(data, str):
        return ""

    return data

shark_attacks["Country"] = shark_attacks["Country"].apply(clean_empty)
shark_attacks["Area"] = shark_attacks["Area"].apply(clean_empty)
shark_attacks["Location"] = shark_attacks["Location"].apply(clean_empty)
shark_attacks = shark_attacks[shark_attacks["Country"].str.match("^.+$")]
shark_attacks = shark_attacks[shark_attacks["Area"].str.match("^.+$")]
shark_attacks = shark_attacks[shark_attacks["Location"].str.match("^.+$")]
shark_attacks

C:\Users\skyli\AppData\Local\Temp\ipykernel_12400\224201197.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_attacks["Country"] = shark_attacks["Country"].apply(clean_empty)
C:\Users\skyli\AppData\Local\Temp\ipykernel_12400\224201197.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_attacks["Area"] = shark_attacks["Area"].apply(clean_empty)
C:\Users\skyli\AppData\Local\Temp\ipykernel_12400\224201197.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

,index,Year,Type,Country,Area,Location,Activity,Unnamed: 9,Age,Injury,Fatal (Y/N),Time,Species,Month,Day
2,2,2020,Unprovoked,AUSTRALIA,New South Wales,Windang Beach,Surfing,M,59,Laceration ot left ankle and foot,N,08h00,"""A small shark""",1,17
3,3,2020,Unprovoked,NEW ZEALAND,Southland,Oreti Beach,Surfing,F,13,Minor injury to lower leg,N,20h30,Broadnose seven gill shark?,1,16
4,4,2020,Unprovoked,USA,North Carolina,"Rodanthe, Dare County",Surfing,M,26,Lacerations to foot,N,14h33,NaN,1,13
5,5,2020,Unprovoked,THAILAND,Phang Nga Province,Nang Thong Beach,Swimming,M,75,Calf bitten,N,07h15,NaN,1,12
6,6,2020,Unprovoked,AUSTRALIA,Queensland,North West Island,Swimming,F,7,Lacerations to leg,N,17h30,Lemon shark,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6237,6237,1837,Unprovoked,AUSTRALIA,New South Wales,Macleay River,Washing his feet,NaN,12,"FATAL Injured by shark, died of tetanus",Y,Evening,NaN,1,17
6249,6249,1830,Unprovoked,USA,Massachusetts,"Swampscott, Essex County","Fishing from dory, shark upset boat & he fell ...",NaN,52,FATAL,Y,NaN,NaN,7,26
6256,6256,1828,Unprovoked,SIERRA LEONE,Western Area,"River Sierra Leone, 35 miles upriver from Free...","British ship, Britannia, was loading lumber. H...",NaN,17,"Left arm severed 2.5"" from elbow, groin, abdom...",N,NaN,NaN,9,28
6274,6274,1817,Unprovoked,SRI LANKA,Western Province,Colombo,Swimming,NaN,22,FATAL,Y,Evening,NaN,5,11


In [727]:
# Step 8: Replace values of "F" with "Y" in column "Fatal (Y/N)".

def clean_fatal(fatal):

    # If already known, skip it.

    if fatal == "Y" or fatal == "N":
        return fatal
    
    # If "F", return "Y".

    if fatal == "F":
        return "Y"
    
    # Some other invalid value (e.g. "M" or "2017" or "NaN")
    
    return "UNKNOWN"

shark_attacks["Fatal (Y/N)"] = shark_attacks["Fatal (Y/N)"].apply(clean_fatal)

shark_attacks

,index,Year,Type,Country,Area,Location,Activity,Unnamed: 9,Age,Injury,Fatal (Y/N),Time,Species,Month,Day
2,2,2020,Unprovoked,AUSTRALIA,New South Wales,Windang Beach,Surfing,M,59,Laceration ot left ankle and foot,N,08h00,"""A small shark""",1,17
3,3,2020,Unprovoked,NEW ZEALAND,Southland,Oreti Beach,Surfing,F,13,Minor injury to lower leg,N,20h30,Broadnose seven gill shark?,1,16
4,4,2020,Unprovoked,USA,North Carolina,"Rodanthe, Dare County",Surfing,M,26,Lacerations to foot,N,14h33,NaN,1,13
5,5,2020,Unprovoked,THAILAND,Phang Nga Province,Nang Thong Beach,Swimming,M,75,Calf bitten,N,07h15,NaN,1,12
6,6,2020,Unprovoked,AUSTRALIA,Queensland,North West Island,Swimming,F,7,Lacerations to leg,N,17h30,Lemon shark,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6237,6237,1837,Unprovoked,AUSTRALIA,New South Wales,Macleay River,Washing his feet,NaN,12,"FATAL Injured by shark, died of tetanus",Y,Evening,NaN,1,17
6249,6249,1830,Unprovoked,USA,Massachusetts,"Swampscott, Essex County","Fishing from dory, shark upset boat & he fell ...",NaN,52,FATAL,Y,NaN,NaN,7,26
6256,6256,1828,Unprovoked,SIERRA LEONE,Western Area,"River Sierra Leone, 35 miles upriver from Free...","British ship, Britannia, was loading lumber. H...",NaN,17,"Left arm severed 2.5"" from elbow, groin, abdom...",N,NaN,NaN,9,28
6274,6274,1817,Unprovoked,SRI LANKA,Western Province,Colombo,Swimming,NaN,22,FATAL,Y,Evening,NaN,5,11


In [728]:
# Step 9: Remove rows where there is no value for the "Fatal (Y/N)" column.
shark_attacks = shark_attacks[shark_attacks["Fatal (Y/N)"].str.match("^Y|N$")]

shark_attacks

,index,Year,Type,Country,Area,Location,Activity,Unnamed: 9,Age,Injury,Fatal (Y/N),Time,Species,Month,Day
2,2,2020,Unprovoked,AUSTRALIA,New South Wales,Windang Beach,Surfing,M,59,Laceration ot left ankle and foot,N,08h00,"""A small shark""",1,17
3,3,2020,Unprovoked,NEW ZEALAND,Southland,Oreti Beach,Surfing,F,13,Minor injury to lower leg,N,20h30,Broadnose seven gill shark?,1,16
4,4,2020,Unprovoked,USA,North Carolina,"Rodanthe, Dare County",Surfing,M,26,Lacerations to foot,N,14h33,NaN,1,13
5,5,2020,Unprovoked,THAILAND,Phang Nga Province,Nang Thong Beach,Swimming,M,75,Calf bitten,N,07h15,NaN,1,12
6,6,2020,Unprovoked,AUSTRALIA,Queensland,North West Island,Swimming,F,7,Lacerations to leg,N,17h30,Lemon shark,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6237,6237,1837,Unprovoked,AUSTRALIA,New South Wales,Macleay River,Washing his feet,NaN,12,"FATAL Injured by shark, died of tetanus",Y,Evening,NaN,1,17
6249,6249,1830,Unprovoked,USA,Massachusetts,"Swampscott, Essex County","Fishing from dory, shark upset boat & he fell ...",NaN,52,FATAL,Y,NaN,NaN,7,26
6256,6256,1828,Unprovoked,SIERRA LEONE,Western Area,"River Sierra Leone, 35 miles upriver from Free...","British ship, Britannia, was loading lumber. H...",NaN,17,"Left arm severed 2.5"" from elbow, groin, abdom...",N,NaN,NaN,9,28
6274,6274,1817,Unprovoked,SRI LANKA,Western Province,Colombo,Swimming,NaN,22,FATAL,Y,Evening,NaN,5,11


In [729]:
# Step 10: Remove empty or indeterminant values in the "Time" column.

def clean_time(time):

    # Replace NaN with empty strings.

    if not isinstance(time, str):
        return ""
    
    # Must match the expected format.

    time = time.strip().strip("\"")

    re_match = re.match("^[0-9]+h[0-9]+$", time)

    # Format matched exactly.

    if re_match is not None:
        return time
    
    # Use averages for word-based times of day.

    if time == 'Midday':
        return "12h00"

    if time == 'Afternoon' or time == 'After noon' or time == 'Early afternoon':
        return "13h30"

    if time == 'Late afternoon':
        return "16h30"
    
    if time == 'Dusk' or time == 'Early evening' or time == 'Sunset':
        return "18h00"

    if time == 'Evening':
        return "19h00"

    if time == 'Night':
        return "20h00"

    if time == 'Dawn':
        return "03h00"

    if time == 'Early morning':
        return "06h00"

    if time == 'Morning':
        return "09h00"
    
    # If the time is solely numeric, reformat it.

    if time.isdigit():

        time_a = ''
        time_b = time[-3]
        time_c = time[-2]
        time_d = time[-1]

        if len(time) == 4:
            time_a = time[-4]
        else:
            time_a = '0'
        
        return time_a + time_b + 'h' + time_c + time_d

    # Did not match expected format, mark for deletion.

    return ""

shark_attacks["Time"] = shark_attacks["Time"].apply(clean_time)
shark_attacks = shark_attacks[~shark_attacks["Time"].str.match("^$")]

shark_attacks

C:\Users\skyli\AppData\Local\Temp\ipykernel_12400\2765263157.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark_attacks["Time"] = shark_attacks["Time"].apply(clean_time)


,index,Year,Type,Country,Area,Location,Activity,Unnamed: 9,Age,Injury,Fatal (Y/N),Time,Species,Month,Day
2,2,2020,Unprovoked,AUSTRALIA,New South Wales,Windang Beach,Surfing,M,59,Laceration ot left ankle and foot,N,08h00,"""A small shark""",1,17
3,3,2020,Unprovoked,NEW ZEALAND,Southland,Oreti Beach,Surfing,F,13,Minor injury to lower leg,N,20h30,Broadnose seven gill shark?,1,16
4,4,2020,Unprovoked,USA,North Carolina,"Rodanthe, Dare County",Surfing,M,26,Lacerations to foot,N,14h33,NaN,1,13
5,5,2020,Unprovoked,THAILAND,Phang Nga Province,Nang Thong Beach,Swimming,M,75,Calf bitten,N,07h15,NaN,1,12
6,6,2020,Unprovoked,AUSTRALIA,Queensland,North West Island,Swimming,F,7,Lacerations to leg,N,17h30,Lemon shark,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6118,6118,1863,Unprovoked,AUSTRALIA,New South Wales,"""Bellynahinch"" on the Manning River",Bathing,NaN,17,FATAL,Y,19h00,NaN,1,10
6151,6151,1858,Unprovoked,AUSTRALIA,Victoria,Hobson Bay,Bathing,NaN,22,FATAL,Y,15h00,NaN,3,14
6237,6237,1837,Unprovoked,AUSTRALIA,New South Wales,Macleay River,Washing his feet,NaN,12,"FATAL Injured by shark, died of tetanus",Y,19h00,NaN,1,17
6274,6274,1817,Unprovoked,SRI LANKA,Western Province,Colombo,Swimming,NaN,22,FATAL,Y,19h00,NaN,5,11


In [730]:
# Step 11: Rename "Species " column to "Species". Yes, there is an extra space.

shark_attacks = shark_attacks.rename(columns={"Species ": "Species"})

shark_attacks

,index,Year,Type,Country,Area,Location,Activity,Unnamed: 9,Age,Injury,Fatal (Y/N),Time,Species,Month,Day
2,2,2020,Unprovoked,AUSTRALIA,New South Wales,Windang Beach,Surfing,M,59,Laceration ot left ankle and foot,N,08h00,"""A small shark""",1,17
3,3,2020,Unprovoked,NEW ZEALAND,Southland,Oreti Beach,Surfing,F,13,Minor injury to lower leg,N,20h30,Broadnose seven gill shark?,1,16
4,4,2020,Unprovoked,USA,North Carolina,"Rodanthe, Dare County",Surfing,M,26,Lacerations to foot,N,14h33,NaN,1,13
5,5,2020,Unprovoked,THAILAND,Phang Nga Province,Nang Thong Beach,Swimming,M,75,Calf bitten,N,07h15,NaN,1,12
6,6,2020,Unprovoked,AUSTRALIA,Queensland,North West Island,Swimming,F,7,Lacerations to leg,N,17h30,Lemon shark,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6118,6118,1863,Unprovoked,AUSTRALIA,New South Wales,"""Bellynahinch"" on the Manning River",Bathing,NaN,17,FATAL,Y,19h00,NaN,1,10
6151,6151,1858,Unprovoked,AUSTRALIA,Victoria,Hobson Bay,Bathing,NaN,22,FATAL,Y,15h00,NaN,3,14
6237,6237,1837,Unprovoked,AUSTRALIA,New South Wales,Macleay River,Washing his feet,NaN,12,"FATAL Injured by shark, died of tetanus",Y,19h00,NaN,1,17
6274,6274,1817,Unprovoked,SRI LANKA,Western Province,Colombo,Swimming,NaN,22,FATAL,Y,19h00,NaN,5,11


In [731]:
# Step 12: Replace empty values for "Species" with "Unknown".

def clean_empty_species(species):
    if not isinstance(species, str):
        return "Unknown"

    return species

shark_attacks["Species"] = shark_attacks["Species"].apply(clean_empty_species)

shark_attacks

,index,Year,Type,Country,Area,Location,Activity,Unnamed: 9,Age,Injury,Fatal (Y/N),Time,Species,Month,Day
2,2,2020,Unprovoked,AUSTRALIA,New South Wales,Windang Beach,Surfing,M,59,Laceration ot left ankle and foot,N,08h00,"""A small shark""",1,17
3,3,2020,Unprovoked,NEW ZEALAND,Southland,Oreti Beach,Surfing,F,13,Minor injury to lower leg,N,20h30,Broadnose seven gill shark?,1,16
4,4,2020,Unprovoked,USA,North Carolina,"Rodanthe, Dare County",Surfing,M,26,Lacerations to foot,N,14h33,Unknown,1,13
5,5,2020,Unprovoked,THAILAND,Phang Nga Province,Nang Thong Beach,Swimming,M,75,Calf bitten,N,07h15,Unknown,1,12
6,6,2020,Unprovoked,AUSTRALIA,Queensland,North West Island,Swimming,F,7,Lacerations to leg,N,17h30,Lemon shark,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6118,6118,1863,Unprovoked,AUSTRALIA,New South Wales,"""Bellynahinch"" on the Manning River",Bathing,NaN,17,FATAL,Y,19h00,Unknown,1,10
6151,6151,1858,Unprovoked,AUSTRALIA,Victoria,Hobson Bay,Bathing,NaN,22,FATAL,Y,15h00,Unknown,3,14
6237,6237,1837,Unprovoked,AUSTRALIA,New South Wales,Macleay River,Washing his feet,NaN,12,"FATAL Injured by shark, died of tetanus",Y,19h00,Unknown,1,17
6274,6274,1817,Unprovoked,SRI LANKA,Western Province,Colombo,Swimming,NaN,22,FATAL,Y,19h00,Unknown,5,11


In [732]:
# Step 13: Clean "Species" column to have only the name of the species.



In [733]:
# Step 14: Renamed "Unnamed: 9" to "Sex of Victim", where values of M, F, and U (unknown) are available.

shark_attacks = shark_attacks.rename(columns={"Unnamed: 9": "Sex of Victim"})

shark_attacks

,index,Year,Type,Country,Area,Location,Activity,Sex of Victim,Age,Injury,Fatal (Y/N),Time,Species,Month,Day
2,2,2020,Unprovoked,AUSTRALIA,New South Wales,Windang Beach,Surfing,M,59,Laceration ot left ankle and foot,N,08h00,"""A small shark""",1,17
3,3,2020,Unprovoked,NEW ZEALAND,Southland,Oreti Beach,Surfing,F,13,Minor injury to lower leg,N,20h30,Broadnose seven gill shark?,1,16
4,4,2020,Unprovoked,USA,North Carolina,"Rodanthe, Dare County",Surfing,M,26,Lacerations to foot,N,14h33,Unknown,1,13
5,5,2020,Unprovoked,THAILAND,Phang Nga Province,Nang Thong Beach,Swimming,M,75,Calf bitten,N,07h15,Unknown,1,12
6,6,2020,Unprovoked,AUSTRALIA,Queensland,North West Island,Swimming,F,7,Lacerations to leg,N,17h30,Lemon shark,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6118,6118,1863,Unprovoked,AUSTRALIA,New South Wales,"""Bellynahinch"" on the Manning River",Bathing,NaN,17,FATAL,Y,19h00,Unknown,1,10
6151,6151,1858,Unprovoked,AUSTRALIA,Victoria,Hobson Bay,Bathing,NaN,22,FATAL,Y,15h00,Unknown,3,14
6237,6237,1837,Unprovoked,AUSTRALIA,New South Wales,Macleay River,Washing his feet,NaN,12,"FATAL Injured by shark, died of tetanus",Y,19h00,Unknown,1,17
6274,6274,1817,Unprovoked,SRI LANKA,Western Province,Colombo,Swimming,NaN,22,FATAL,Y,19h00,Unknown,5,11


In [734]:
# Step 15: Replace NaN values in Sex of Victim to U.

def clean_sex_of_victim(sex):
    if not isinstance(sex, str):
        return "U"
    
    if sex != "M" and sex != "F":
        return "U"

    return sex


shark_attacks["Sex of Victim"] = shark_attacks["Sex of Victim"].apply(clean_sex_of_victim)

shark_attacks

,index,Year,Type,Country,Area,Location,Activity,Sex of Victim,Age,Injury,Fatal (Y/N),Time,Species,Month,Day
2,2,2020,Unprovoked,AUSTRALIA,New South Wales,Windang Beach,Surfing,M,59,Laceration ot left ankle and foot,N,08h00,"""A small shark""",1,17
3,3,2020,Unprovoked,NEW ZEALAND,Southland,Oreti Beach,Surfing,F,13,Minor injury to lower leg,N,20h30,Broadnose seven gill shark?,1,16
4,4,2020,Unprovoked,USA,North Carolina,"Rodanthe, Dare County",Surfing,M,26,Lacerations to foot,N,14h33,Unknown,1,13
5,5,2020,Unprovoked,THAILAND,Phang Nga Province,Nang Thong Beach,Swimming,M,75,Calf bitten,N,07h15,Unknown,1,12
6,6,2020,Unprovoked,AUSTRALIA,Queensland,North West Island,Swimming,F,7,Lacerations to leg,N,17h30,Lemon shark,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6118,6118,1863,Unprovoked,AUSTRALIA,New South Wales,"""Bellynahinch"" on the Manning River",Bathing,U,17,FATAL,Y,19h00,Unknown,1,10
6151,6151,1858,Unprovoked,AUSTRALIA,Victoria,Hobson Bay,Bathing,U,22,FATAL,Y,15h00,Unknown,3,14
6237,6237,1837,Unprovoked,AUSTRALIA,New South Wales,Macleay River,Washing his feet,U,12,"FATAL Injured by shark, died of tetanus",Y,19h00,Unknown,1,17
6274,6274,1817,Unprovoked,SRI LANKA,Western Province,Colombo,Swimming,U,22,FATAL,Y,19h00,Unknown,5,11


In [735]:
# Export new table as CSV.

#shark_attacks.to_csv("clean_GSAF5.xls.csv")